In [2]:
'''Train a simple deep CNN on the CIFAR10 small images dataset.

It gets to 75% validation accuracy in 25 epochs, and 79% after 50 epochs.
(it's still underfitting at that point, though).
'''

from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
from train.train_model import Train


In [3]:

train = Train()
batch_size = 4
num_classes = 2
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = train.start_training()

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
# y_train = keras.utils.to_categorical(y_train, num_classes)
# y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train[:10])

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


GPU not found


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 256, 256, 32)      896       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 254, 254, 32)      9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 127, 127, 32)      0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 127, 127, 64)      18496     
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 125, 125, 64)      36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 62, 62, 64)        0         
__________

Epoch 1/200


StopIteration: unsupported operand type(s) for /: 'NoneType' and 'int'

In [1]:
from train.train_model import Train

train = Train()
train.start_training()

/home/vozman/projects/slides/slide-analysis-nn/venv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


GPU not found


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 256, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 254, 254, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 127, 127, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 127, 127, 64)      18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 125, 125, 64)      36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 62, 62, 64)        0         
__________

Epoch 1/200


  1/894 [..............................] - ETA: 3:31:51 - loss: 0.7152 - acc: 0.3438

...


Epoch 00001: val_loss improved from inf to 0.54840, saving model to /home/vozman/projects/slides/slide-analysis-nn/train/snapshots/train_17/slide_analysis_01_0.55.h5


894/894 [==============================] - 12632s 14s/step - loss: 0.3691 - acc: 0.8585 - val_loss: 0.5484 - val_acc: 0.7287


Epoch 2/200


  1/894 [..............................] - ETA: 3:19:56 - loss: 0.3981 - acc: 0.8750

KeyboardInterrupt: 

In [3]:
from prediction.predict import Predict

predict_example = Predict()
predicted_results = predict_example.predict_slide('/home/vozman/projects/slides/slide-analysis-nn/train/datasets/source/small_with_tumor_images/Tumor_044.tif_61274:136934:69235:146478.tif')

[[   0    0]
 [ 256    0]
 [ 512    0]
 ...
 [7424 9472]
 [7680 9472]
 [7936 9472]]
predict
0:38
[0.0959467887878418]
1:38
[0.0959467887878418, 0.06082797050476074]
2:38


[0.0959467887878418, 0.06082797050476074, 0.07087850570678711]
3:38
[0.0959467887878418, 0.06082797050476074, 0.07087850570678711, 0.14611029624938965]
4:38


[0.0959467887878418, 0.06082797050476074, 0.07087850570678711, 0.14611029624938965, 0.12292075157165527]
5:38
[0.0959467887878418, 0.06082797050476074, 0.07087850570678711, 0.14611029624938965, 0.12292075157165527, 0.1139976978302002]
6:38


[0.0959467887878418, 0.06082797050476074, 0.07087850570678711, 0.14611029624938965, 0.12292075157165527, 0.1139976978302002, 0.15430879592895508]
7:38
[0.0959467887878418, 0.06082797050476074, 0.07087850570678711, 0.14611029624938965, 0.12292075157165527, 0.1139976978302002, 0.15430879592895508, 0.12135648727416992]
8:38


[0.0959467887878418, 0.06082797050476074, 0.07087850570678711, 0.14611029624938965, 0.12292075157165527, 0.1139976978302002, 0.15430879592895508, 0.12135648727416992, 0.09510588645935059]
9:38
[0.0959467887878418, 0.06082797050476074, 0.07087850570678711, 0.14611029624938965, 0.12292075157165527, 0.1139976978302002, 0.15430879592895508, 0.12135648727416992, 0.09510588645935059, 0.0847930908203125]
10:38


...

0:38
[0.0959467887878418, 0.06082797050476074, 0.07087850570678711, 0.14611029624938965, 0.12292075157165527, 0.1139976978302002, 0.15430879592895508, 0.12135648727416992, 0.09510588645935059, 0.0847930908203125, 0.13875365257263184, 0.11624836921691895, 0.12740492820739746, 0.11200213432312012, 0.1307539939880371, 0.12903833389282227, 0.15857529640197754, 0.08994412422180176, 0.11399388313293457, 0.1103672981262207, 0.08962821960449219, 0.11564445495605469, 0.08691763877868652, 0.11414027214050293, 0.11491823196411133, 0.09549069404602051, 0.0992746353149414, 0.13657426834106445, 0.10058188438415527, 0.08601927757263184, 0.08842182159423828, 0.1452183723449707, 0.08668398857116699, 0.0869743824005127, 0.1134030818939209, 0.08727741241455078, 0.1107640266418457, 0.09425115585327148, 0.09181451797485352]
1:38
[0.0959467887878418, 0.06082797050476074, 0.07087850570678711, 0.14611029624938965, 0.12292075157165527, 0.1139976978302002, 0.15430879592895508, 0.12135648727416992, 0.09510588645

[0.0959467887878418, 0.06082797050476074, 0.07087850570678711, 0.14611029624938965, 0.12292075157165527, 0.1139976978302002, 0.15430879592895508, 0.12135648727416992, 0.09510588645935059, 0.0847930908203125, 0.13875365257263184, 0.11624836921691895, 0.12740492820739746, 0.11200213432312012, 0.1307539939880371, 0.12903833389282227, 0.15857529640197754, 0.08994412422180176, 0.11399388313293457, 0.1103672981262207, 0.08962821960449219, 0.11564445495605469, 0.08691763877868652, 0.11414027214050293, 0.11491823196411133, 0.09549069404602051, 0.0992746353149414, 0.13657426834106445, 0.10058188438415527, 0.08601927757263184, 0.08842182159423828, 0.1452183723449707, 0.08668398857116699, 0.0869743824005127, 0.1134030818939209, 0.08727741241455078, 0.1107640266418457, 0.09425115585327148, 0.09181451797485352, 0.0993499755859375, 0.07514619827270508]
3:38
[0.0959467887878418, 0.06082797050476074, 0.07087850570678711, 0.14611029624938965, 0.12292075157165527, 0.1139976978302002, 0.15430879592895508

[0.0959467887878418, 0.06082797050476074, 0.07087850570678711, 0.14611029624938965, 0.12292075157165527, 0.1139976978302002, 0.15430879592895508, 0.12135648727416992, 0.09510588645935059, 0.0847930908203125, 0.13875365257263184, 0.11624836921691895, 0.12740492820739746, 0.11200213432312012, 0.1307539939880371, 0.12903833389282227, 0.15857529640197754, 0.08994412422180176, 0.11399388313293457, 0.1103672981262207, 0.08962821960449219, 0.11564445495605469, 0.08691763877868652, 0.11414027214050293, 0.11491823196411133, 0.09549069404602051, 0.0992746353149414, 0.13657426834106445, 0.10058188438415527, 0.08601927757263184, 0.08842182159423828, 0.1452183723449707, 0.08668398857116699, 0.0869743824005127, 0.1134030818939209, 0.08727741241455078, 0.1107640266418457, 0.09425115585327148, 0.09181451797485352, 0.0993499755859375, 0.07514619827270508, 0.06049084663391113, 0.07024836540222168, 0.060491323471069336, 0.10169053077697754]
7:38
[0.0959467887878418, 0.06082797050476074, 0.070878505706787

[0.0959467887878418, 0.06082797050476074, 0.07087850570678711, 0.14611029624938965, 0.12292075157165527, 0.1139976978302002, 0.15430879592895508, 0.12135648727416992, 0.09510588645935059, 0.0847930908203125, 0.13875365257263184, 0.11624836921691895, 0.12740492820739746, 0.11200213432312012, 0.1307539939880371, 0.12903833389282227, 0.15857529640197754, 0.08994412422180176, 0.11399388313293457, 0.1103672981262207, 0.08962821960449219, 0.11564445495605469, 0.08691763877868652, 0.11414027214050293, 0.11491823196411133, 0.09549069404602051, 0.0992746353149414, 0.13657426834106445, 0.10058188438415527, 0.08601927757263184, 0.08842182159423828, 0.1452183723449707, 0.08668398857116699, 0.0869743824005127, 0.1134030818939209, 0.08727741241455078, 0.1107640266418457, 0.09425115585327148, 0.09181451797485352, 0.0993499755859375, 0.07514619827270508, 0.06049084663391113, 0.07024836540222168, 0.060491323471069336, 0.10169053077697754, 0.06282186508178711, 0.06276416778564453, 0.07412028312683105]
1

157.64708423614502 sup


In [4]:
predicted_results


(array([0, 0, 0, ..., 0, 0, 0]),
 array([0.9341314 , 0.9349627 , 0.9017763 , ..., 0.80786467, 0.80808276,
        0.6042391 ], dtype=float32))